> You know, Apple Computers started in a garage, and we are starting in a condo. So we already have a leg up on Apple. - Pam Beesly


Today started as peaceful morning in Scraton Pennsylvania. There has been a long time vacated position ever since Ryan was fired as VP.
Everyone from the office just learnt that Charles Miner, previously worked at Saticoy Steel, is joining Dunder Mifflin as new VP.

Let's add Charles to Dunder Mifflin's scim group.


In [ ]:
# First we need to import the scim client and configuration
from sync_app.client import ClientConfiguration, SCIMClient

# scim client configuration require server URL and access token.
# The server URL is the base URL of the SCIM server.
server_url = "https://api.scim.dev/scim/v2/"
# The access token is the token that is used to authenticate the client to the SCIM server.
# You can get the access token from the website
# Visit https://scim.dev/apikey/ to get your token
auth_token = "YOUR_ACCESS_CODE"


# Now we have everything we need to create a scim client configuration
scim_dev_client_config = ClientConfiguration(
    server_url=server_url, auth_token=auth_token
)

# Now we can create a scim client using the configuration
scim_dev_client = SCIMClient(scim_dev_client_config)

In [ ]:
# This block of code will check if the group already exists and delete it if it does
from sync_app.client import Filter

response = scim_dev_client.get_groups(
    filter=Filter(filter='displayName eq "Dunder Mifflin"')
).to_dict()
if response["totalResults"] != 0:
    group_id = response["Resources"][0]["id"]
    scim_dev_client.delete_group(group_id)
####

from scim_helper import course_setup

# You can ignore this part for now, the crouse_setup function create new group and add bunch exist users to the group.
dunder_mifflin_group_id, user_ids = course_setup(auth_token)

In [ ]:
# You can take a look at the group_id and user_ids
print("Group ID is the UUID for the group created by the course_setup helper function")
print(dunder_mifflin_group_id)
print(
    "It also create a nice dictionary of user_id to user_name for the users added to the group."
)
user_ids

Now the course set up is finished, let's try couple basic scim request!


In [ ]:
# The first thing we can do is to get all users from the SCIM server

# users =
users.to_dict()

# This will return ALL the users from server.

In [ ]:
# You can also just fetch one user if you have the user's id

# user =

user.to_dict()

By now, you might notice that every response from server will begin with `schemas`.
It discribe what the response `meta` looks like.
So client to fetch the schemas to have better understand on each meta attribute.


In [ ]:
# You can also fetch the schemas from the server
# The get_full_schemas make a get request to `/v2/Schema` endpoint
full_schema = scim_dev_client.get_full_schemas()
full_schema.to_dict()

In [ ]:
# You can also fetch user or group schema from the server
# The get_user_schemas make a get request to `/v2/Schemas/urn:ietf:params:scim:schemas:core:2.0:User` endpoint
user_schemas = scim_dev_client.get_user_schemas()
user_schemas.to_dict()

In [ ]:
# You can also fetch user or group schema from the server
# The get_group_schemas make a get request to `/v2/Schemas/urn:ietf:params:scim:schemas:core:2.0:Group` endpoint

####
# scim.dev group schema currently does not work even with their playground ui requests.
####

# group_schemas = scim_dev_client.get_group_schemas()
# group_schemas.to_dict()

Enough of buzy work, now let's create a new user object!

Here is what User object required!

```
    def __init__(
        self,
        meta: ResourceMeta,
        user_id: UUID,
        displayName: Optional[str],
        active: bool,
        userName: str,
        name: Optional[Name],
        emails: Optional[Iterable[Email]],
        groups: Optional[Iterable[ResourceRef]] = None,
    ):
```

We can go over them one by one.


The first thing we need is some meta data!
ResourceMeta does provide create_meta function to create THE meta

```
def create_meta(resource_type: ResourceType, location: str) -> "ResourceMeta":
```

It requires the resource type and the resource URL
There are five different type resource that exist

- User
- Group
- ServiceProviderConfig
- ResourceType
- Schema

Looks like the one we need right now is `User` because duh, we are creating a user object.

But what the hack is location?
Resource Location is the endpoint where a resource could be re-fetched from the same service prodiver.
Since we (Duo) is storing the user, the location would be `https://duo.com/Users/<uuid>`


In [ ]:
from sync_app.scim.resource import ResourceMeta

# We would also need some uuid
from uuid import uuid5, NAMESPACE_X500

# uuid5 is used to generate a unique identifier from a name and a namespace
# We would use the NAMESPACE_X500 as the namespace
# So we could use DN as the name to generate the uuid

# uuid =

meta = ResourceMeta.create_meta(
    # <Resource type>
    # <Resource location>
)

Looking back on User Object

```
    def __init__(
        self,
        meta: ResourceMeta,
        user_id: UUID,
        displayName: Optional[str],
        active: bool,
        userName: str,
        name: Optional[Name],
        emails: Optional[Iterable[Email]],
        groups: Optional[Iterable[ResourceRef]] = None,
    ):
```

We actually killed two bird with one stone since meta require uuid.
`DisplayName` is The name of the User, suitable for display to end-users. The name SHOULD be the full name of the User being described, if known. So it should be like `Charles Miner`
`active` is A Boolean value indicating the User's administrative status. So `True` should be it.
`userName` is Unique identifier for the User, typically used by the user to directly authenticate to the service provider. Each User MUST include a non-empty userName value. This identifier MUST be unique across the service provider's entire set of Users. REQUIRED. So `cminer` should be fine.

`Name` is a werid one, because it contain some sub attribute.
`Name` Object looks like this, it's all optional but we fill familyName and givenName easily.

```
@dataclass
class Name:
    formatted: Optional[str] = None
    familyName: Optional[str] = None
    givenName: Optional[str] = None
```

`emails` is also a object that contain subattribute.

```
@dataclass
class Email:
    value: Optional[str] = None
    type: Optional[str] = None
    primary: Optional[bool] = None
    display: Optional[str] = None
```

`value` is email address
`type` is for work/home/other/etc
`primary` is if the email is the preferred one.
`display` is the the thing make least sense, so we will skip that for now.

`groups` is use set the group for the user, for now we are going to use the exist Dunder Mifflin Group!


In [ ]:
# Now let's create a user!
from sync_app.scim.user import User, Name, Email
from sync_app.scim.resource import ResourceRef

cminer = User(
    # meta
    # user_id
    # displayName
    # active
    # userName
    # name
    # emails
    # groups
)

# Now we can try to push the user to the server!
response = scim_dev_client.create_user(cminer)
response.to_dict()

In [ ]:
# We should also keep track of the newly created user to our user_ids dictionary
user_ids["Charles.Miner"] = response.to_dict()["id"]
user_ids

### Optional scim step

Before we sent user object to scim SCP, we could validate it against the user schemas that we receieved earlier.
This is optinal to the course.


In [ ]:
from jsonschema import validate

validate(cminer.to_dict(), user_schemas.to_dict())
# This should return None if the user is valid

Now we have Charles in the system, Charles the new VP finally able to access his new laptop and all customer information for the first day!


Even Micheal had good time with Charles in the beginning, the tension between Micheal and Charles is so thich you can pratical cut it with a butter knife.

This result Micheal put his 2 week notice by the end of day. Micheal approved multiple Dunder Mifflin employee to join Micheal to his new paper company!


After goofy around for 2 week, the final day at Dunder Mifflin for Micheal has arrived. There is no one willing to quit the job at Dunder Mifflin and join Micheal for his new adventure.


Let's remove Micheal from the Dunder Mifflin group, so he can no long access inportant company secrets.


#### SCIM knowledge: _When you ask scim server to delete an user, scim server might not actual delete the user. But will not return the user information when you ask about the user again. So it might be a better idea to soft delete an user to make it inactive._


In [ ]:
# Lets first deactive Micheal's status
from sync_app.scim.patchop import PatchOp, Operation

deactive_patch_op = PatchOp(
    # <operation>
)

response = scim_dev_client.update_user(
    # <user_id>
    # <patch_op>
)
response.to_dict()

In [ ]:
# Let's try to remove Micheal from the dunder mifflin group

> Confidence-- it's the food of the wise man, but the liquor of the fool. --Vikram


Oh no, Micheal Scott Paper Company just started.
We need to do some scim work to keep the scim record up to dates.
Here are list of things we need to.

- Create a new group called Micheal Scott Paper Company.
- Add Micheal, Pam, Ryan to the new Group.
- Deactive Pam from Dunder Mifflin group since she is no long working for Dunder Mifflin.


In [ ]:
# Similar to before, this block of code will check if the group already exists and delete it if it does
from sync_app.client import Filter

response = scim_dev_client.get_groups(
    filter=Filter(filter='displayName eq "Micheal Scott Paper Company"')
).to_dict()
if response["totalResults"] != 0:
    group_id = response["Resources"][0]["id"]
    scim_dev_client.delete_group(group_id)
####


# Create a new group
from sync_app.scim.group import Group
from sync_app.scim.resource import ResourceMeta
from uuid import uuid5, NAMESPACE_X500

micheal_scott_paper_company = Group(
    # displayName
    # group_id
    # meta
)

response = scim_dev_client.create_group(micheal_scott_paper_company)

micheal_scott_paper_company_id = response.to_dict()["id"]

response.to_dict()

In [ ]:
# Now let's try to do multiple operations in one patch call
# Let's create a patch operation to add all three people to the new group
from sync_app.scim.patchop import PatchOp, Operation

add_three_peope_op = PatchOp(
    # <operation>
)

response = scim_dev_client.update_group(
    # <group_id>
    # <patch_op>
)

response.to_dict()

In [ ]:
# Now let's deactive Pam from dunder mifflin group
# We have done the similar thing for Micheal Scott before.
from sync_app.scim.patchop import PatchOp, Operation

remove_patch_op = PatchOp(
    # <operation>
)

response = scim_dev_client.update_group(
    # <group_id>
    # <patch_op>
)

response.to_dict()

Now we have created the group and populated some users in the group. Let's verify we have done everything correctly.
There are mutiple ways we can group information.


In [ ]:
# The esiast way to get a group or user is to use the get_user or get_group method along with the id

# For example, this is how you get the original Dunder Mifflin group

response = scim_dev_client.get_group(micheal_scott_paper_company_id)

response.to_dict()

In [ ]:
# But it might be really hard to track the id of the user or group you want to get.
# Or you lost the id of the user or group you want to get.
# You could also use filter to filter the user or group you want to get.
# Now let's try to get the user we just created but with filter.

# First we need to create a new filter
from sync_app.client import Filter

filter = Filter(filter='displayName eq "Dunder Mifflin"')
response = scim_dev_client.get_groups(filter=filter)

response.to_dict()

In [ ]:
# The response from get_groups does not contain all the information about the group.
# You can add attribute that you tell server on what attribute you want to get.
response = scim_dev_client.get_group(
    micheal_scott_paper_company_id, attributes=["members"]
)
response.to_dict()

In [ ]:
# You could also get the same result if you want to use user endpoint instead groups endpoint.

from sync_app.client import Filter

filter = Filter(filter='groups.value eq "{}"'.format(micheal_scott_paper_company_id))
response = scim_dev_client.get_users(filter=filter)
response.to_dict()

#### SCIM knowledge: _You could query user attribute too!_

However, the get_user call we have today already return all the attribute we have in those exmaple users. So it will not alter the result, but for more complex users in the real world, you could add attribute to ask server to return more attributes than the default ones.


Looks like we also should update their emails! Let's do that next.


In [ ]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    # <operation>
)

response = scim_dev_client.update_user(
    # <user_id>
    # <patch_op>
)

response.to_dict()

In [ ]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    # <operation>
)

response = scim_dev_client.update_user(
    # <user_id>
    # <patch_op>
)

response.to_dict()

In [ ]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    # <operation>
)

response = scim_dev_client.update_user(
    # <user_id>
    # <patch_op>
)

response.to_dict()

#### Extra scim knowledges:

- You could also uses `PUT` calls to replace exist user and group. So instead of update one attribute, you will replace entire entry with new user/group. The main benefit for `PUT` call is you could save the same UUID.
- ... (more to come)


WooHoo! Now our job is done! Feel free to try more operition!
